In [4]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import datetime

# Incorporate data
df = pd.read_csv('data.csv')

# Create Date Columns
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.date
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.strftime('%H:%M')

df.dtypes

_id                          int64
Timestamp           datetime64[ns]
Redemption Count             int64
Sales Count                  int64
Year                         int32
Month                        int32
Day                         object
Hour                         int32
Minute                      object
dtype: object

In [26]:
from dash import dcc, html, Input, Output, State, Dash
import plotly.express as px
import pandas as pd
import numpy as np
import calendar

# Incorporate data
df = pd.read_csv('data.csv')

# Create Date Columns
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month.apply(lambda x: calendar.month_name[int(x)])
df['Day'] = df['Timestamp'].dt.date
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.strftime('%H:%M')



app = Dash(__name__)

app.layout = html.Div([
    html.H2("Ferry Redemptions Drilldown"),
    html.Button("Back", id="back-button", n_clicks=0),
    dcc.Graph(id="bar-graph")
])

# App state to track drilldown level
drill_levels = ['Year', 'Month', 'Day', 'Hour', 'Minute']
drill_titles = [
    'Total Redemptions by Year',
    'Monthly Redemptions in {}',
    'Daily Redemptions in {} {}',
    'Hourly Redemptions on {}',
    'Minute-Level Redemptions on {}'
]

drill_state = []

@app.callback(
    Output("bar-graph", "figure"),
    Input("bar-graph", "clickData"),
    Input("back-button", "n_clicks"),
    State("bar-graph", "figure")
)
def update_graph(clickData, n_clicks, current_fig):
    global drill_state
    ctx = dash.callback_context

    if not ctx.triggered:
        trigger = None
    else:
        trigger = ctx.triggered[0]['prop_id'].split('.')[0]

    if trigger == "back-button" and drill_state:
        drill_state.pop()

    elif trigger == "bar-graph" and clickData is not None and len(drill_state) < 4:
        clicked_value = str(clickData['points'][0]['x'])
        drill_state.append(clicked_value)

    # Determine drilldown level
    level = len(drill_state)

    if level == 0:
        df_grouped = df.groupby("Year")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped, x="Year", y="Redemption Count", title=drill_titles[0])
        fig.update_xaxes(type='category')
        return fig

    elif level == 1:
        df_filtered = df[df['Year'] == int(drill_state[0])]
        df_grouped = df_filtered.groupby("Month")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped.sort_values(by="Month"), x="Month", y="Redemption Count", title=drill_titles[1].format(drill_state[0]))
        fig.update_xaxes(type='category')
        return fig

    elif level == 2:
        df_filtered = df[(df['Year'] == int(drill_state[0])) & (df['Month'] == drill_state[1])]
        df_grouped = df_filtered.groupby("Day")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped.sort_values(by="Day"), x="Day", y="Redemption Count", title=drill_titles[2].format(drill_state[1], drill_state[0]))
        fig.update_xaxes(type='category')
        return fig

    elif level == 3:
        df_filtered = df[df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()]
        df_grouped = df_filtered.groupby("Hour")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped, x="Hour", y="Redemption Count", title=drill_titles[3].format(drill_state[2]))
        fig.update_xaxes(type='category')
        return fig

    elif level == 4:
        df_filtered = df[
            (df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()) &
            (df['Hour'] == int(drill_state[3]))
    ]
        df_grouped = df_filtered.groupby("Timestamp")['Redemption Count'].sum().reset_index()
        df_grouped["label"] = df_grouped["Timestamp"].dt.strftime('%H:%M')
        fig = px.bar(df_grouped, x="label", y="Redemption Count", title=drill_titles[4].format(drill_state[2]))
        fig.update_xaxes(type='category')
        return fig

    return dash.no_update

if __name__ == '__main__':
    app.run(debug=True)


In [25]:
df

,_id,Timestamp,Redemption Count,Sales Count,Year,Month,Day,Hour,Minute
0,1,2025-04-11 18:45:00,5,4,2025,4,2025-04-11,18,18:45
1,2,2025-04-11 18:30:00,11,45,2025,4,2025-04-11,18,18:30
2,3,2025-04-11 18:15:00,9,7,2025,4,2025-04-11,18,18:15
3,4,2025-04-11 18:00:00,25,14,2025,4,2025-04-11,18,18:00
4,5,2025-04-11 17:45:00,3,2,2025,4,2025-04-11,17,17:45
...,...,...,...,...,...,...,...,...,...
242207,242208,2015-05-04 16:00:00,0,2,2015,5,2015-05-04,16,16:00
242208,242209,2015-05-01 16:00:00,1,0,2015,5,2015-05-01,16,16:00
242209,242210,2015-05-01 15:45:00,0,1,2015,5,2015-05-01,15,15:45
242210,242211,2015-05-01 15:15:00,0,2,2015,5,2015-05-01,15,15:15


In [58]:

# Compare to day object
df_filtered = df[
    (df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()) &
    (df['Hour'] == int(drill_state[3]))
]
df_grouped = df_filtered.groupby("Timestamp")['Redemption Count'].sum().reset_index()
df_grouped["label"] = df_grouped["Timestamp"].dt.strftime('%H:%M')
fig = px.bar(df_grouped, x="label", y="Redemption Count", title=drill_titles[4].format(drill_state[2]))
fig.update_xaxes(type='category')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'label=%{x}<br>Redemption Count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['11:00', '11:15', '11:30', '11:45'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'Kns/RA==', 'dtype': 'i1'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Minute-Level Redemptions on 2016-05-12'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'label'}, 'type': 'category'},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Redemption Count'}}}
})

In [61]:
drill_state

['2018', '5', '2018-05-13', '12']

In [28]:
df_filtered = df[(df['Year'] == int(drill_state[0])) & (df['Month'] == int(drill_state[1]))]
df_grouped = df_filtered.groupby("Day")['Redemption Count'].sum().reset_index()
df_grouped

,Day,Redemption Count
0,2018-05-01,3206
1,2018-05-02,2510
2,2018-05-03,1186
3,2018-05-04,1150
4,2018-05-05,6815
5,2018-05-06,2446
6,2018-05-07,2474
7,2018-05-08,2821
8,2018-05-09,2484
9,2018-05-10,1703


In [21]:
df.dtypes

_id                          int64
Timestamp           datetime64[ns]
Redemption Count             int64
Sales Count                  int64
Year                         int32
Month                        int32
Day                         object
Hour                         int32
Minute                      object
dtype: object

In [91]:
import datetime
datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()

datetime.date(2023, 5, 12)

In [85]:
drill_state[2]

'2023-05-12'

In [80]:
type(data['Day'].iloc[0])

datetime.date

In [69]:
data['Day'].iloc[0]
dt_test = pd.to_datetime(data['Day'].iloc[0])
dt_test

Timestamp('2022-01-01 00:00:00')

In [52]:
df_filtered = data[(data['Year'] == 2023) & (data['Month'] == '2023-05')]
df_grouped = df_filtered.groupby("Day")['redemptions'].sum().reset_index()
df_grouped

,Day,redemptions
0,2023-05-01,1853
1,2023-05-02,1914
2,2023-05-03,1935
3,2023-05-04,1905
4,2023-05-05,1908
5,2023-05-06,1907
6,2023-05-07,1981
7,2023-05-08,1868
8,2023-05-09,1949
9,2023-05-10,1926


In [51]:
data.dtypes

timestamp      datetime64[ns]
redemptions             int32
Year                    int32
Month                  object
Day                    object
Hour                    int32
Minute                 object
dtype: object

In [96]:
data.to_csv('testing_data.csv', index=False)